In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emoji

In [4]:
emoji_dictionary={
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":downcast_face_with_sweat:",
    "4":":fork_and_knife:"
}

In [5]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😓
🍴


In [36]:
train=pd.read_csv("train_emoji.csv",header=None)
test=pd.read_csv("test_emoji.csv",header=None)
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [37]:

data=train.values


In [38]:
for i in range(10):
    print(data[i][0])

never talk to me again
I am proud of your achievements
It is the worst day in my life
Miss you so much
food is life
I love you mum
Stop saying bullshit
congratulations on your acceptance
The assignment is too long 
I want to go play


In [39]:
for i in range(10):
    print(emoji.emojize(emoji_dictionary[str(data[i][1])]))

😓
😃
😓
❤️
🍴
❤️
😓
😃
😓
⚾


In [44]:
from keras.utils import to_categorical
XT=train[0]
YT=to_categorical(train[1])
Xt=test[0]
Yt=to_categorical(test[1])

In [45]:
XT.shape
YT.shape

(132, 5)

In [47]:
f=open('glove.6B.50d.txt',encoding='utf-8')
embedding_index={}
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()


In [48]:
embedding_index["eat"]

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01],
      dtype=float32)

In [49]:
def embedding_output(X):
    maxLen=10
    #emb_dim=embedding_index["eat"].shape[0]
    embedding_out=np.zeros((X.shape[0],maxLen,50))
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for ij in range(len(X[ix])):
            embedding_out[ix][ij]=embedding_index[X[ix][ij].lower()]
    return embedding_out

In [50]:
embedding_matrix_train=embedding_output(XT)
embedding_matrix_test=embedding_output(Xt)

<ipython-input-49-df15d4713c94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix]=X[ix].split()


In [51]:
print(embedding_matrix_train.shape)
print(embedding_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [52]:
from keras.models import Sequential
from keras.layers import *

In [56]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(embedding_matrix_train,YT,epochs=40,batch_size=32,shuffle=True,validation_split=0.2)

Epoch 1/40
4/4 [==============================] - 1s 153ms/step - loss: 1.6284 - accuracy: 0.1714 - val_loss: 1.5951 - val_accuracy: 0.1852
Epoch 2/40
4/4 [==============================] - 0s 13ms/step - loss: 1.5826 - accuracy: 0.3048 - val_loss: 1.5925 - val_accuracy: 0.2222
Epoch 3/40
4/4 [==============================] - 0s 15ms/step - loss: 1.5575 - accuracy: 0.3429 - val_loss: 1.5915 - val_accuracy: 0.2222
Epoch 4/40
4/4 [==============================] - 0s 13ms/step - loss: 1.5066 - accuracy: 0.3905 - val_loss: 1.5894 - val_accuracy: 0.2222
Epoch 5/40
4/4 [==============================] - 0s 14ms/step - loss: 1.4701 - accuracy: 0.4476 - val_loss: 1.5797 - val_accuracy: 0.2222
Epoch 6/40
4/4 [==============================] - 0s 12ms/step - loss: 1.4304 - accuracy: 0.4381 - val_loss: 1.5699 - val_accuracy: 0.1852
Epoch 7/40
4/4 [==============================] - 0s 12ms/step - loss: 1.3823 - accuracy: 0.4190 - val_loss: 1.5564 - val_accuracy: 0.2222
Epoch 8/40
4/4 [==========

In [58]:
pred=model.predict_classes(embedding_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [60]:
for i in range(Xt.shape[0]):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
he did not answer
😓
he got a raise
😃
she got me a present
😃
ha ha ha it was so funny
😃
he is a good friend
😃
I am upset
😓
We had such a lovely dinner tonight
😃
where is the food
🍴
Stop making this joke ha ha ha
😃
where is the ball
⚾
work is hard
😃
This girl is messing with me
❤️
are you serious ha ha
❤️
Let us go play baseball
⚾
This stupid grader is not working
😓
work is horrible
😃
Congratulation for having a baby
😃
stop messing around
😓
any suggestions for dinner
🍴
I love taking breaks
❤️
you brighten my day
❤️
I boiled rice
🍴
she is a bully
❤️
Why are you feeling bad
😓
I am upset
😓
I worked during my birthday
😃
My grandmother is the love of my life
❤️
enjoy your break
⚾
valentine day is near
😃
I miss you so much
❤️
throw the ball
⚾
My life is so boring
❤️
she said yes
😃
will you be my valentine
❤️
he can pitch really well
😃
dance with me
😃
I am starving
😓
See you at the restaurant
🍴
I like to laugh
😃
I will go dance
😃
I like your jacket
❤️
i miss her
❤️
what is your 